In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix
import nltk
import string
from collections import Counter
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer
from collections import Counter
from scipy.sparse import csr_matrix
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [2]:
train_data = pd.read_csv("train.dat", sep = '\t', header = None)
test_data = pd.read_csv("test.dat", sep = '\t', header = None)

train_data.columns = ['abf','peptide']
test_data.columns = ['peptide']
train_arr = [p for p in train_data.peptide]
train_length = len(train_arr)
train_lbl = [l for l in train_data.abf]
test_arr = [p for p in test_data.peptide]

train = train_arr + test_arr
label = train_lbl
print(train[0:2])
print (label[0:2])

['DVELDLVEISPNALP', 'KADEELFNKLFFGT']
[-1, -1]


In [3]:
def cmer(train, c):
    r""" Given a name and parameter c, return the vector of c-mers associated with the name
    """
    if len(train) < c:
        return [train]
    v = []
    for i in range(len(train)-c+1):
        v.append(train[i:(i+c)])
    return v

train = [cmer(p,2) for p in train]
train
print(train)

[['DV', 'VE', 'EL', 'LD', 'DL', 'LV', 'VE', 'EI', 'IS', 'SP', 'PN', 'NA', 'AL', 'LP'], ['KA', 'AD', 'DE', 'EE', 'EL', 'LF', 'FN', 'NK', 'KL', 'LF', 'FF', 'FG', 'GT'], ['FL', 'LV', 'VA', 'AL', 'LH', 'HL', 'LG', 'GT', 'TA', 'AF', 'FA', 'AL', 'LL', 'LW', 'WY', 'YF', 'FR', 'RK', 'KR', 'RW', 'WC', 'CA', 'AL', 'LV', 'VR', 'RG', 'GF', 'FF', 'FA', 'AS', 'SF', 'FG', 'GG', 'GR', 'RR', 'RN', 'ND', 'DD', 'DA', 'AH', 'HM', 'MM'], ['RD', 'DQ', 'QM', 'MR', 'RA', 'AR', 'RI', 'IA', 'AD', 'DI', 'IT', 'TG', 'GV', 'VA', 'AI', 'IS', 'SR', 'RI', 'IA'], ['RK', 'KR', 'RL', 'LQ', 'QL', 'LL', 'LL', 'LL'], ['PG', 'GF', 'FC', 'CV', 'VG', 'GE', 'EA', 'AS', 'SP', 'PL', 'LK', 'KS', 'SP', 'PG', 'GR', 'RR', 'RE', 'EL', 'LG', 'GH', 'HG', 'GN', 'NL', 'LA'], ['TP', 'PE', 'EE', 'ED', 'DT', 'TM', 'MV', 'VM', 'MK', 'KP', 'PL', 'LP', 'PP', 'PE', 'EP', 'PA', 'AP'], ['SK', 'KD', 'DM', 'MR', 'RV', 'VQ', 'QR'], ['IA', 'AA', 'AS', 'SI', 'IR', 'RH', 'HP', 'PL', 'LH', 'HC', 'CT'], ['HP', 'PN', 'NL', 'LN', 'NL', 'LF', 'FA', 'AA', 'A

In [4]:
def build_matrix(docs):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    idx = {}
    tid = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat, idx

def csr_info(mat, name="", non_empy=False):
    r""" Print out info about this CSR matrix. If non_empy, 
    report number of non-empty rows and cols as well
    """
    if non_empy:
        print("%s [nrows %d (%d non-empty), ncols %d (%d non-empty), nnz %d]" % (
                name, mat.shape[0], 
                sum(1 if mat.indptr[i+1] > mat.indptr[i] else 0 
                for i in range(mat.shape[0])), 
                mat.shape[1], len(np.unique(mat.indices)), 
                len(mat.data)))
    else:
        print( "%s [nrows %d, ncols %d, nnz %d]" % (name, 
                mat.shape[0], mat.shape[1], len(mat.data)) )

train_mat, idx = build_matrix(train)
csr_info(train_mat,non_empy=True)

 [nrows 1958 (1958 non-empty), ncols 413 (413 non-empty), nnz 41645]


/var/folders/sn/jv8q19ts7p1cd4qlt53d6w800000gn/T/ipykernel_41594/1817023932.py:18: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  ind = np.zeros(nnz, dtype=np.int)
/var/folders/sn/jv8q19ts7p1cd4qlt53d6w800000gn/T/ipykernel_41594/1817023932.py:20: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your curre

In [5]:
def csr_idf(mat, copy=False, **kargs):
    r""" Scale a CSR matrix by idf. 
    Returns scaling factors as dict. If copy is True, 
    returns scaled matrix and scaling factors.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # document frequency
    df = defaultdict(int)
    for i in ind:
        df[i] += 1
    # inverse document frequency
    for k,v in df.items():
        df[k] = np.log(nrows / float(v))  ## df turns to idf - reusing memory
    # scale by idf
    for i in range(0, nnz):
        val[i] *= df[ind[i]]
        
    return df if copy is False else mat

def csr_l2normalize(mat, copy=False, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat

In [6]:
train_mat1 = csr_idf(train_mat, copy=True)
train_mat2 = csr_l2normalize(train_mat1, copy=True)
print("train_mat2:", train_mat2[15,:20].todense(),"\n")
print(train_mat2.shape)

train_mat2: [[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.09923768
  0.         0.         0.         0.         0.         0.
  0.         0.        ]] 

(1958, 413)


In [7]:
train_mat = train_mat2[:train_length]
test_mat = train_mat2[train_length:]
print("test_mat:", test_mat[15,:20].todense(),"\n")
print(test_mat.shape)

test_mat: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]] 

(392, 413)


In [8]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
svd.fit_transform(train_mat2)

array([[ 0.24153091, -0.11069061, -0.04684923, -0.05908607,  0.07161789],
       [ 0.19953091, -0.01889382,  0.17667938,  0.01060982,  0.06553804],
       [ 0.29302383,  0.0848866 , -0.02603964,  0.18036437, -0.012598  ],
       ...,
       [ 0.3446799 , -0.02961649, -0.15856314,  0.07310838,  0.09659442],
       [ 0.26669755, -0.07498132, -0.02464575, -0.04693078, -0.01688346],
       [ 0.28722918,  0.03314219,  0.07715039, -0.03805429,  0.11135961]])

In [9]:
from sklearn import svm
from sklearn.svm import LinearSVC


#clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf = svm.SVC()
#clf = make_pipeline(StandardScaler(with_mean=False), SVC(kernel = "linear", C=100.0))
clf.fit(train_mat, train_lbl)
pred_test = clf.predict(test_mat) 

In [10]:
with open('final_prediction123.dat','w+') as file:
    for p in pred_test:
        file.write(str(p) +"\n")